# 13장 분류예측분석

## 13.2 선형판별분석
### 분석 시나리오

A쇼핑에서는 고객이탈을 효과적으로 예측하고자 고객들의 거래이력 데이터를 활용하여 선형판별모형 기반의 이탈예측 모형을 개발하고자 한다. 독립변수로는 방문빈도, 1회 평균매출액, 거래기간을 사용하기로 하였다. 이를 가설은 다음과 같이 구성할 수 있다.  

#### H0 (귀무가설)= 방문빈도와 1회 평균 매출액,거래기간은 이탈에 영향을 주지 않는다.
#### H1 (연구가설)= 방문빈도와 1회 평균매출액,거래기간은 이탈에 영향을 준다. 


In [1]:
#1. 모듈 및 데이터 탑재
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix
df = pd.read_csv('Ashopping.csv', sep=',', encoding='CP949')

#2. 종속변수와 독립변수 구분하기
X=df[["방문빈도", "1회_평균매출액", "거래기간"]]
Y=df[["이탈여부"]]

#4. 선형판별분석
lda = LDA().fit(X, Y) #독립변수, 종속변수

#5. 예측결과
print('판별식 선형계수 : ',np.round(lda.coef_,3))
print('\n판별식 절편 : ',np.round(lda.intercept_,3))
print('\n예측결과 : \n', pd.DataFrame(lda.predict(X)))
print('\n예측스코어 : \n', pd.DataFrame(lda.predict_proba(X)))
print('\n예측 정확도 :', lda.score(X, Y))

#1. 분류행렬표 출력
cf_m=pd.DataFrame(confusion_matrix(Y, lda.predict(X)))
cf_m.columns=['예측 0','예측 1']
cf_m.index=['실제 0','실제 1']
print('\n분류행렬표 : \n', cf_m)


판별식 선형계수 :  [[ 0.086  0.    -0.096]]

판별식 절편 :  [72.916]

예측결과 : 
      0
0    0
1    1
2    0
3    0
4    0
..  ..
995  0
996  0
997  0
998  0
999  0

[1000 rows x 1 columns]

예측스코어 : 
                 0             1
0    1.000000e+00  2.208135e-13
1    1.370801e-10  1.000000e+00
2    1.000000e+00  1.999582e-13
3    1.000000e+00  7.951807e-11
4    1.000000e+00  1.309479e-11
..            ...           ...
995  1.000000e+00  2.283614e-12
996  1.000000e+00  1.727200e-10
997  1.000000e+00  3.609038e-14
998  1.000000e+00  8.248046e-13
999  1.000000e+00  6.555998e-13

[1000 rows x 2 columns]

예측 정확도 : 0.986

분류행렬표 : 
       예측 0  예측 1
실제 0   921    12
실제 1     2    65


C:\Users\Seungwoo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 13.3 로지스틱 회귀분석
### 분석 시나리오

앞의 선형판별분석과 마찬가지로 이탈 예측모형을 만들기 위해 거래기간, Recency, 성별을 사용하고자 한다. 여기에서 독립변수 중 성별은 명목척도이므로 로지스틱 회귀분석을 사용하는 것이 바람직하다. 이를 위한 가설은 다음과 같이 설정할 수 있다.

#### H0 (귀무가설)= 거래기간,Recency,성별은 이탈에 영향을 주지 않는다.
#### H1 (연구가설)= 거래기간,Recency,성별은 이탈에 영향을 준다.

In [5]:
#1. 모듈 및 데이터 탑재
import pandas as pd
import numpy as np
import statsmodels.api as sm
pd.options.display.float_format = '{:.3f}'.format

df = pd.read_csv('Ashopping.csv', sep=',', encoding='CP949')

#2. 더미변수 생성하기
df2=pd.get_dummies(df['성별'], prefix='성별', drop_first=False)
df3=pd.concat([df,df2],axis=1)

#3. 종속변수와 독립변수 구분하기
df3['intercept'] = 1.0
x = df3[['intercept','거래기간', 'Recency', '성별_0']]
y = df3[['이탈여부']]

#4. 로지스틱 회귀분석 실행하기
logit = sm.Logit(y,x).fit( )

#5. 분석결과 출력하기
print(logit.summary2( ))
print('\n오즈비 :\n', np.exp(logit.params))
cf_m2=pd.DataFrame(logit.pred_table())
cf_m2.columns=['예측 0','예측 1']
cf_m2.index=['실제 0','실제 1']
print('\n분류행렬표 : \n', cf_m2)


Optimization terminated successfully.
         Current function value: 0.015583
         Iterations 12
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.937     
Dependent Variable: 이탈여부             AIC:              39.1656   
Date:               2020-06-16 14:29 BIC:              58.7966   
No. Observations:   1000             Log-Likelihood:   -15.583   
Df Model:           3                LL-Null:          -245.81   
Df Residuals:       996              LLR p-value:      1.7725e-99
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     12.0000                                      
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
intercept     19.1980    4.0839   4.7009  0.0000  11.1937  27.2022
거래기간          -0.0204    0.0052  -3.9300  0.0001  -0.0306  -0